In [3]:
%pip install --upgrade --quiet devtools

Note: you may need to restart the kernel to use updated packages.


In [13]:
from dotenv import load_dotenv
import json
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

load_dotenv()

True

In [14]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="mps")

In [15]:
filename = "./Chatbot_data_filtered.json"
pc = Pinecone()
index = pc.Index("noon-catalog")

i = j = 0
batch_size = 100
upserted_count = 0
texts, ids, embeds, metadata = [], [], [], []
with open(filename) as file:
    for line in file:
        if j % 1000 == 0:
            print(f"Iteration: {j}. Upserted: ~{upserted_count}")
        if i % batch_size == 0 and texts:
            embeds = model.encode(texts)
            upserted_count += len(embeds)
            index.upsert(vectors=zip(ids, embeds, metadata))
            texts, ids, embeds, metadata = [], [], [], []

        doc = json.loads(line.rstrip())
        if (
            doc.get("title_en")
            and doc.get("description")
            and doc.get("highlights")
            and doc.get("offer_price")
            and doc.get("msrp")
            and doc.get("image_url")
            and doc.get("product_url")
        ):
            ids.append(f'{doc["offer_code"]}_{i}')
            m = {**doc}
            del m["description"]
            del m["title_en"]
            del m["highlights"]
            text = f'Title: {doc["title_en"]}\nDescription: {doc["description"]}.\nHighlights: {doc["highlights"]}\nPrice: {doc.get("offer_price", doc.get("msrp", "Unavailable"))}\nImageURL: {doc["image_url"]}\nProductURL: {doc["product_url"]}'
            m["text"] = text
            metadata.append({**m})
            texts.append(text)
            i += 1
        j += 1

Iteration: 0. Upserted: ~0
Iteration: 1000. Upserted: ~400
Iteration: 2000. Upserted: ~800
Iteration: 3000. Upserted: ~1300
Iteration: 4000. Upserted: ~1800
Iteration: 5000. Upserted: ~2100
Iteration: 6000. Upserted: ~2700
Iteration: 7000. Upserted: ~3200
Iteration: 8000. Upserted: ~3700
Iteration: 9000. Upserted: ~4300
Iteration: 10000. Upserted: ~5000
Iteration: 11000. Upserted: ~5800
Iteration: 12000. Upserted: ~6500
Iteration: 13000. Upserted: ~7400
Iteration: 14000. Upserted: ~8100
Iteration: 15000. Upserted: ~8500
Iteration: 16000. Upserted: ~9000
Iteration: 17000. Upserted: ~9600
Iteration: 18000. Upserted: ~10100
Iteration: 19000. Upserted: ~10800
Iteration: 20000. Upserted: ~11400
Iteration: 21000. Upserted: ~12000
Iteration: 22000. Upserted: ~12800
Iteration: 23000. Upserted: ~13400
Iteration: 24000. Upserted: ~13800
Iteration: 25000. Upserted: ~14400
Iteration: 26000. Upserted: ~14700
Iteration: 27000. Upserted: ~15100
Iteration: 28000. Upserted: ~15700
Iteration: 29000. Ups